In [1]:
from __future__ import division, absolute_import, print_function

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import tensorflow as tf
tf.__version__

'1.13.1'

# Detecting lines of 5 with a ConvNet and hand-woven features

In [12]:
hor=np.zeros([5,5], dtype=float)
hor[2]=1
diag=np.eye(5, dtype=float)
filters = np.array([hor, hor.T, diag, diag[::-1]])
kernel_init = tf.constant_initializer(np.rollaxis(filters, 0, 3))
bias_init = tf.constant_initializer(-4.)

## Take particular note of the shape: Channels last
np.shape(kernel_init.value) 

(5, 5, 4)

### Verifying  the function with some examples

In [13]:
boards = np.zeros([6, 10, 10])
for i in range(5):
    boards[0][5][3+i] = 1.
    boards[1][3+i][5] = 1.
    boards[2][8-i][3+i] = 1.
    boards[3][2+i][2+i] = 1.
    boards[4][2+i][2+i] = 1.
    boards[5][2+i][2+i] = 1.
boards[0]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 1., 1., 1., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [14]:
inp=tf.constant(boards.reshape(-1,10, 10, 1))
out = tf.layers.conv2d(kernel_size=5, kernel_initializer=kernel_init, 
                       filters=4, inputs=inp, padding='same', 
                       bias_initializer=bias_init, activation='relu')
out = tf.layers.max_pooling2d(inputs=out, pool_size=10, strides=1)

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    res = sess.run(out)
res = np.squeeze(np.rollaxis(res, -1, 0))
print(res)

[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 1.]
 [0. 0. 1. 0. 0. 0.]]


You see that every sample (the six columns) has a $1$ at the position that corresponds to the particular pattern that has been recognized.

In [15]:
sum(res)

array([1., 1., 1., 1., 1., 1.])

### Creating labels with the hand-crafted features

The *labels* graph maps each sample that contains a line of 5 to a $1$, all others to a $0$

In [16]:
inp_heuristics = tf.placeholder(name="inp_heuristics", shape=[None, 10, 10, 1], dtype=tf.float32)
out = tf.layers.conv2d(kernel_size=5, kernel_initializer=kernel_init, 
                       filters=4, inputs=inp_heuristics, padding='same', 
                       bias_initializer=bias_init, activation='relu')
out = tf.layers.max_pooling2d(inputs=out, pool_size=10, strides=1)
labels = tf.squeeze(tf.sign(tf.reduce_sum(out, axis=3)))

In [17]:
samples = (np.random.uniform(size = [5, 10,10]) < .3).astype(float).reshape(-1,10,10,1)

In [18]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    res = sess.run(labels, feed_dict={inp_heuristics: samples})
res

array([0., 0., 1., 1., 1.], dtype=float32)

In [19]:
np.rollaxis(samples[3],-1, 0)

array([[[0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 1., 0., 0., 0.],
        [1., 0., 0., 1., 0., 0., 1., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1., 0., 1., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 1., 1., 1., 0., 0., 1., 0., 0.],
        [1., 0., 0., 1., 1., 0., 1., 1., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 1., 0., 0.]]])

In [20]:
def create_samples(sess, placeholder, n=100):
    samples = (np.random.uniform(size = [n, 10,10]) < .3).astype(float).reshape(-1,10,10,1)
    lbls = sess.run(labels, feed_dict={placeholder: samples})
    return samples, lbls

### Some kind of ResNet

Concat over the channels rather than simply adding the residual connection (just feels better!)

In [76]:
class ResNet:
    def __init__(self, n_blocks):
        self.inps = tf.placeholder(name="inp_resnet", shape=[None, 10, 10, 1], dtype=tf.float32)
        self.lbls = tf.placeholder(name="lbl_resnet", shape=[None, 1], dtype=tf.float32)

        out = self.inps

        for i in range(n_blocks):
            out = self._res_block(out)

        out = tf.layers.max_pooling2d(inputs=out, pool_size=10, strides=1)
        out = tf.reduce_sum(out, axis=3)
        self.out = tf.reshape(tensor=out, shape=[-1, 1])
        
        self.errors = (self.lbls - self.out)**2
        self.accuracy=tf.reduce_sum(tf.cast(self.errors < .1, dtype=tf.int64))

        self.loss = tf.losses.mean_squared_error(self.out, self.lbls)
        self.optimizer = tf.train.AdamOptimizer(learning_rate=1e-4)
        self.trainer = self.optimizer.minimize(self.loss)
    
    
    def _res_block(self, inp, filters=16, kernel_size=3, activation='elu'):
        bn1 = inp #tf.layers.batch_normalization(inp)
        out1 = tf.layers.conv2d(kernel_size=kernel_size, filters=filters, inputs=bn1, padding='same', activation=activation)
        bn2 = out1 #tf.layers.batch_normalization(out1)
        out2 = tf.layers.conv2d(kernel_size=kernel_size, filters=filters, inputs=bn2, padding='same', activation=activation)
        return tf.concat([inp, out2], axis=3)

In [77]:
resnet = ResNet(5)

In [78]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for i in range(20001):
        smp, lbl = create_samples(sess, inp_heuristics, 100)
        lbl = lbl.reshape([-1, 1])
        l, o, _ = sess.run([resnet.loss, resnet.out, resnet.trainer], feed_dict={resnet.lbls: lbl, resnet.inps: smp})
        if i % 200 == 0:
            print("training loss %s: " % l)
            #print(o, lbl)
            
    smp, lbl = create_samples(sess, inp_heuristics, 20)
    lbl = lbl.reshape([-1, 1])
    acc, pred, err = sess.run([resnet.accuracy, resnet.out, resnet.errors], feed_dict={resnet.inps: smp, resnet.lbls: lbl})
    print("Accuracy %s" % acc)
    print(list(zip(pred, lbl)))


training loss 938.0239: 
training loss 0.7217333: 
training loss 0.4154512: 
training loss 0.4266182: 
training loss 0.3923775: 
training loss 0.32861468: 
training loss 0.3353776: 
training loss 0.2655028: 
training loss 0.27668983: 
training loss 0.28840914: 
training loss 0.23260888: 
training loss 0.22413684: 
training loss 0.23979336: 
training loss 0.24748448: 
training loss 0.18262994: 
training loss 0.23833355: 
training loss 0.17571072: 
training loss 0.21711713: 
training loss 0.16091911: 
training loss 0.22697644: 
training loss 0.18443593: 
training loss 0.22739184: 
training loss 0.20181984: 
training loss 0.17909949: 
training loss 0.17921434: 
training loss 0.17410707: 
training loss 0.2004467: 
training loss 0.1900327: 
training loss 0.1667454: 
training loss 0.19191477: 
training loss 0.17100452: 
training loss 0.17860237: 
training loss 0.1632259: 
training loss 0.162935: 
training loss 0.15124765: 
training loss 0.15620966: 
training loss 0.1856036: 
training loss 0.

KeyboardInterrupt: 

In [58]:
lbl

array([[1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]], dtype=float32)

In [28]:
with tf.Session() as sess: 
    sess.run(init)
    smp, lbl = create_samples(sess, inp_heuristics, 10)
    lbl = lbl.reshape([-1, 1])
    l = sess.run(resnet.loss, feed_dict={resnet.lbls: lbl, resnet.inps: smp})
    print(l)

557.4439


In [59]:
pred, lbl

(array([[ 0.77217877],
        [ 0.65571237],
        [ 0.05552252],
        [ 0.06998692],
        [ 0.03211707],
        [ 0.08071324],
        [ 0.07653587],
        [ 0.07904088],
        [ 0.00248633],
        [-0.0495755 ]], dtype=float32), array([[1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]], dtype=float32))

In [14]:
n_blocks=10

inp_resnet = tf.placeholder(name="inp_resnet", shape=[None, 10, 10, 1], dtype=tf.float32)
out = inp_resnet

for i in range(n_blocks):
    out = res_block(out)

out = tf.layers.max_pooling2d(inputs=out, pool_size=10, strides=1)
out = tf.reduce_sum(out, axis=3)
out = tf.reshape(tensor=out, shape=[-1, 1])

Instructions for updating:
Use keras.layers.batch_normalization instead.


In [30]:
resnet = ResNet(10)

AttributeError: 'ResNet' object has no attribute 'lbl_ph'

In [15]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    res = sess.run(out, feed_dict={inp_resnet: samples})

In [16]:
res.shape

(5, 1)

In [17]:
with tf.Session() as sess:
    sess.run(init)
    smp, lbl = create_samples(sess, inp_heuristics, 5)
smp.shape, lbl

((5, 10, 10, 1), array([0., 0., 0., 1., 0.], dtype=float32))

In [18]:
errors = (lbl_ph - out)**2
accuracy=tf.reduce_sum(tf.cast(errors < .1,dtype=tf.int64))

loss = tf.losses.mean_squared_error(out, lbl_ph)
optimizer = tf.train.AdamOptimizer(learning_rate=1e-3)
trainer = optimizer.minimize(loss)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


In [19]:
smp.shape

(5, 10, 10, 1)

In [21]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for i in range(101):
        smp, lbl = create_samples(sess, inp_heuristics, 100)
        lbl = lbl.reshape([-1, 1])
        l, _ = sess.run([loss, trainer], feed_dict={lbl_ph: lbl, inp_resnet: smp})
        if i % 10 == 0:
            print(l)
            
    smp, lbl = create_samples(sess, inp_heuristics, 10)
    lbl = lbl.reshape([-1, 1])
    acc, pred, err = sess.run([accuracy, out, errors], feed_dict={inp_resnet: smp, lbl_ph: lbl})
    print("Accuracy %s" % acc)
    

640.756
40.822334
12.348616
5.978667
3.748187
2.4962647
2.0127873
1.5186
1.5438575
1.2487758
1.1082757
Accuracy 2


In [23]:
pred

array([[1.1214346],
       [1.1950278],
       [1.2284412],
       [1.1846426],
       [1.2190171],
       [1.1985679],
       [1.1330221],
       [1.23429  ],
       [1.2529943],
       [1.2266933]], dtype=float32)

In [24]:
pred, lbl,err

(array([[1.1214346],
        [1.1950278],
        [1.2284412],
        [1.1846426],
        [1.2190171],
        [1.1985679],
        [1.1330221],
        [1.23429  ],
        [1.2529943],
        [1.2266933]], dtype=float32), array([[0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.]], dtype=float32), array([[1.2576154 ],
        [1.4280915 ],
        [0.0521854 ],
        [1.403378  ],
        [1.4860028 ],
        [1.4365649 ],
        [1.283739  ],
        [1.5234718 ],
        [0.06400611],
        [1.5047764 ]], dtype=float32))

In [67]:
with tf.Session() as sess:
    sess.run(init)
    smp, lbl = create_samples(sess, inp_heuristics, 100)
    lbl = lbl.reshape([-1, 1])

In [68]:
lbl

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],